<a href="https://colab.research.google.com/github/JinyongShin/PythonStudy/blob/master/Pytorch/01_DataLoader_and_Customizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

파이토치 제공 데이터 사용

In [2]:
transf = tr.Compose([tr.Resize(8),tr.ToTensor()])

###   파이토치 홈페이지에 전처리에 대한 내용이 있다고합니다. ###
###   이 코드는 8by8 텐서로 변환하는 전처리 코드입니다.      ###
###   PIL Images 에만 적용 가능하다고 합니다. ㅎㅎ           ###

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./',train=True,download=True,transform=transf)
testset = torchvision.datasets.CIFAR10(root='./',train=False,download=True,transform=transf)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
trainset[0][0].size()

torch.Size([3, 8, 8])

In [5]:
trainloader = DataLoader(trainset,batch_size=50,shuffle=True,num_workers=2)
testloader = DataLoader(testset,batch_size=50,shuffle=True,num_workers=2)

In [6]:
len(trainloader)

1000

In [7]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [8]:
images.size()

torch.Size([50, 3, 8, 8])

개인 데이터 사용

In [15]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))

print(train_images.shape,train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [16]:
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.x_data = self.x_data.permute(0,3,1,2) #이미지 수 , 채널 수, 이미지 너비, 높이
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [17]:
train_data = TensorData(train_images,train_labels)
train_loader = DataLoader(train_data, batch_size = 10, shuffle=True)

In [18]:
train_data[0][0].size()

torch.Size([3, 32, 32])

In [20]:
dataiter=iter(train_loader)
images,labels = dataiter.next()

In [22]:
images.size()

torch.Size([10, 3, 32, 32])

# ###from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self):

    def __getitem__(self):

    def __len__(self):


In [23]:
class MyDataset(Dataset):

    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)

    def __getitem__(self,index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        return self.len

In [24]:
class ToTensor:
    def __call__(self,sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        return inputs, torch.LongTensor(labels)

In [25]:
class LinearTensor:
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias

    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope * inputs + self.bias
        return inputs, labels

In [26]:
trans = tr.Compose([ToTensor(),LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1,batch_size=10, shuffle=True)

In [28]:
first_data = ds1[0]
features, labels = first_data
print(type(features),type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [29]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

In [30]:
images1

tensor([[[[489., 233., 227.,  ..., 487.,  81., 121.],
          [369., 305., 115.,  ..., 267., 377., 219.],
          [ 39.,   5., 103.,  ...,  77., 489., 213.],
          ...,
          [355., 235., 217.,  ..., 291., 453., 387.],
          [ 25., 373., 103.,  ..., 167.,  75., 397.],
          [159., 103., 435.,  ..., 201., 355., 461.]],

         [[145., 153., 139.,  ...,  11., 143., 417.],
          [ 27.,  57., 119.,  ...,  73., 131.,  43.],
          [235., 129., 245.,  ..., 269., 143., 249.],
          ...,
          [ 73.,  53., 421.,  ..., 419., 361., 129.],
          [ 99., 193., 235.,  ..., 271., 217., 121.],
          [163., 233., 473.,  ...,  27., 341., 269.]],

         [[507., 145., 331.,  ...,  81., 473.,  63.],
          [371., 317., 403.,  ...,  65.,  59., 155.],
          [435.,  73.,  39.,  ...,  45., 497., 317.],
          ...,
          [389., 263., 151.,  ..., 151., 133.,  87.],
          [275., 377., 245.,  ..., 135., 189.,  79.],
          [383., 433., 259.,  ...